In [1]:
import configparser
import math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os

from tqdm import tqdm

config = configparser.ConfigParser()
config.read('../config.ini', encoding='utf-8')
SAVE_DIR = config.get('settings','SAVE_DIR')
WORK_DIR = config.get('settings','WORK_DIR')
SHARE_DIR = config.get('settings','SHARE_DIR')

settings = configparser.ConfigParser()
settings.read('../settings.ini', encoding='utf-8')
MIN_YEAR = int(settings.get('experiment','MIN_YEAR'))
MAX_YEAR = int(settings.get('experiment','MAX_YEAR'))
RESOLUTION = float(settings.get('experiment','RESOLUTION'))
NMIN = int(settings.get('experiment','NMIN'))

In [3]:
# os.makedirs('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers')

# 2021 valid reader

In [6]:
papers_2016_validread = pd.read_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/papers_2016_all_validread.pickle')

print(papers_2016_validread.shape)
_ = papers_2016_validread[papers_2016_validread['cumulative_reader_2018'].notna()]
print(len(_))
_ = _[_['cumulative_reader_2021'].notna()]
print(len(_)) # 85.4%にreadershipがある

# check_validで新規追加、2018年にreadがなければ除く
_ = _[_['cumulative_reader_2021']!=0].copy()
print(len(_))

_.to_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_2016_all_validread.pickle')

(2290742, 34)
2290742
2290742
2184682


# Matching

In [7]:
papers_2016_validread = pd.read_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_2016_all_validread.pickle')

## citation + field

In [8]:
# slowとlow(2018では同じ)
papers_slow_lowmatched2018 = []
papers_low_slowmatched2018 = []
# slowとfast(2021では同じ)
papers_slow_fastmatched2021 = []
papers_fast_slowmatched2021 = []

for partition in tqdm(sorted(papers_2016_validread['partition_1e-06'].unique())):
    df_slow = papers_2016_validread[(papers_2016_validread['label_hml_2018'].isin(['middle','bottom']))&(papers_2016_validread['label_hml_2021'].isin(['top']))].query(f'`partition_1e-06` == {partition}')
    df_fast = papers_2016_validread[(papers_2016_validread['label_hml_2018'].isin(['top']))&(papers_2016_validread['label_hml_2021'].isin(['top']))].query(f'`partition_1e-06` == {partition}')
    df_low = papers_2016_validread[(papers_2016_validread['label_hml_2018'].isin(['middle','bottom']))&(papers_2016_validread['label_hml_2021'].isin(['middle','bottom']))].query(f'`partition_1e-06` == {partition}')
    
    print(partition, len(df_slow), len(df_fast), len(df_low))
    
    # 2018で揃えたslow vs low
    for c2018, count in df_slow['c_history_sum_2018'].value_counts().items():
        n_sample = min([len(df_slow.query('c_history_sum_2018 == @c2018')),len(df_low.query('c_history_sum_2018 == @c2018'))])
        papers_slow_lowmatched2018.append(df_slow.query('c_history_sum_2018 == @c2018').sample(n_sample, random_state=42))
        papers_low_slowmatched2018.append(df_low.query('c_history_sum_2018 == @c2018').sample(n_sample, random_state=42))
    
    # 2021で揃えたslow vs fast
    for c2021, count in df_slow['c_history_sum'].value_counts().items():
        n_sample = min([len(df_slow.query('c_history_sum == @c2021')),len(df_fast.query('c_history_sum == @c2021'))])
        papers_slow_fastmatched2021.append(df_slow.query('c_history_sum == @c2021').sample(n_sample, random_state=42))
        papers_fast_slowmatched2021.append(df_fast.query('c_history_sum == @c2021').sample(n_sample, random_state=42))

papers_slow_lowmatched2018 = pd.concat(papers_slow_lowmatched2018)
papers_low_slowmatched2018 = pd.concat(papers_low_slowmatched2018)
papers_slow_fastmatched2021 = pd.concat(papers_slow_fastmatched2021)
papers_fast_slowmatched2021 = pd.concat(papers_fast_slowmatched2021)

print(papers_slow_lowmatched2018.shape)
print(papers_low_slowmatched2018.shape)
print(papers_slow_fastmatched2021.shape)
print(papers_fast_slowmatched2021.shape)

papers_slow_lowmatched2018.to_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_slow_1e-06_lowmatched2018.pickle')
papers_low_slowmatched2018.to_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_low_1e-06_slowmatched2018.pickle')
papers_slow_fastmatched2021.to_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_slow_1e-06_fastmatched2021.pickle')
papers_fast_slowmatched2021.to_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_fast_1e-06_slowmatched2021.pickle')

  1%|█▋                                                                                                                | 1/67 [00:00<00:51,  1.28it/s]

-1.0 0 0 0
0.0 5551 22663 228302


  3%|███▍                                                                                                              | 2/67 [00:02<01:22,  1.26s/it]

1.0 4310 15822 158446


  4%|█████                                                                                                             | 3/67 [00:03<01:27,  1.36s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 66/67 [01:08<00:00,  1.12it/s]

64.0 60 160 1369


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [01:09<00:00,  1.03s/it]

65.0 49 284 2512


(54086, 34)
(54086, 34)
(50311, 34)
(50311, 34)


In [16]:
display(papers_slow_lowmatched2018.describe()[['c_history_sum_2018','c_history_sum','jif']])
display(papers_low_slowmatched2018.describe()[['c_history_sum_2018','c_history_sum','jif']])
display(papers_slow_fastmatched2021.describe()[['c_history_sum_2018','c_history_sum','jif']])
display(papers_fast_slowmatched2021.describe()[['c_history_sum_2018','c_history_sum','jif']])

,c_history_sum_2018,c_history_sum,jif
count,54086.000000,54086.000000,54086.000000
mean,9.770477,40.185371,3.421924
std,4.398704,16.379881,2.714791
min,0.000000,5.000000,0.000000
25%,7.000000,30.000000,1.676324
50%,9.000000,38.000000,2.993827
75%,12.000000,48.000000,4.522152
max,23.000000,458.000000,39.592593


,c_history_sum_2018,c_history_sum,jif
count,54086.000000,54086.000000,54086.000000
mean,9.770477,21.023537,3.129720
std,4.398704,10.181996,2.467788
min,0.000000,0.000000,0.000000
25%,7.000000,14.000000,1.469565
50%,9.000000,20.000000,2.766234
75%,12.000000,27.000000,4.198971
max,23.000000,56.000000,84.988372


,c_history_sum_2018,c_history_sum,jif
count,50311.000000,50311.000000,50311.000000
mean,9.938403,41.108883,3.478764
std,4.365069,16.119232,2.738520
min,0.000000,5.000000,0.000000
25%,7.000000,31.000000,1.749280
50%,9.000000,39.000000,3.050293
75%,12.000000,49.000000,4.568182
max,23.000000,458.000000,39.592593


,c_history_sum_2018,c_history_sum,jif
count,50311.000000,50311.000000,50311.000000
mean,17.838584,41.108883,4.111291
std,7.317558,16.119232,3.057137
min,3.000000,5.000000,0.000000
25%,13.000000,31.000000,2.226107
50%,16.000000,39.000000,3.471370
75%,22.000000,49.000000,5.255511
max,145.000000,458.000000,39.592593


## citation + field + journal

In [9]:
papers_slow_lowmatched2018_journal = []
papers_low_slowmatched2018_journal = []
papers_slow_fastmatched2021_journal = []
papers_fast_slowmatched2021_journal = []

matched_eid_low_slowmatched = set()
matched_eid_fast_slowmatched = set()

for partition in tqdm(sorted(papers_2016_validread['partition_1e-06'].unique())):
    df_slow = papers_2016_validread[(papers_2016_validread['label_hml_2018'].isin(['middle','bottom']))&(papers_2016_validread['label_hml_2021'].isin(['top']))].query(f'`partition_1e-06` == {partition}')
    df_fast = papers_2016_validread[(papers_2016_validread['label_hml_2018'].isin(['top']))&(papers_2016_validread['label_hml_2021'].isin(['top']))].query(f'`partition_1e-06` == {partition}')
    df_low = papers_2016_validread[(papers_2016_validread['label_hml_2018'].isin(['middle','bottom']))&(papers_2016_validread['label_hml_2021'].isin(['middle','bottom']))].query(f'`partition_1e-06` == {partition}')
            
# 2018で揃えたslow vs low
    for eid, c2018, journal in df_slow[['eid', 'c_history_sum_2018','journal']].values:
        df_low_target = df_low.query('c_history_sum_2018 == @c2018').query('journal == @journal')
        if len(df_low_target) != 0:
            papers_slow_lowmatched2018_journal.append(df_slow.query('eid == @eid'))
            row = df_low_target.sample(1, random_state=42)
            papers_low_slowmatched2018_journal.append(row)
            matched_eid_low_slowmatched.add(row['eid'].values[0])        
        
    # 2021で揃えたslow vs fast
    for eid, c2021, journal in df_slow[['eid', 'c_history_sum','journal']].values:
        df_fast_target = df_fast.query('c_history_sum == @c2021').query('journal == @journal')
        if len(df_fast_target) != 0:
            papers_slow_fastmatched2021_journal.append(df_slow.query('eid == @eid'))
            row = df_fast_target.sample(1, random_state=42)
            papers_fast_slowmatched2021_journal.append(row)
            matched_eid_fast_slowmatched.add(row['eid'].values[0])

papers_slow_lowmatched2018_journal = pd.concat(papers_slow_lowmatched2018_journal)
papers_low_slowmatched2018_journal = pd.concat(papers_low_slowmatched2018_journal)
papers_slow_fastmatched2021_journal = pd.concat(papers_slow_fastmatched2021_journal)
papers_fast_slowmatched2021_journal = pd.concat(papers_fast_slowmatched2021_journal)

print(papers_slow_lowmatched2018_journal.shape)
print(papers_low_slowmatched2018_journal.shape)
print(papers_slow_fastmatched2021_journal.shape)
print(papers_fast_slowmatched2021_journal.shape)

papers_slow_lowmatched2018_journal.to_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_slow_1e-06_lowmatched2018_journal.pickle')
papers_low_slowmatched2018_journal.to_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_low_1e-06_slowmatched2018_journal.pickle')
papers_slow_fastmatched2021_journal.to_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_slow_1e-06_fastmatched2021_journal.pickle')
papers_fast_slowmatched2021_journal.to_pickle('/disks/qnap2/data/t-miura/2023_readership/final_paper/check1_valid_readers/papers_fast_1e-06_slowmatched2021_journal.pickle')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [07:23<00:00,  6.61s/it]


(40101, 34)
(40101, 34)
(20561, 34)
(20561, 34)


In [17]:
display(papers_slow_lowmatched2018_journal.describe()[['c_history_sum_2018','c_history_sum','jif']])
display(papers_low_slowmatched2018_journal.describe()[['c_history_sum_2018','c_history_sum','jif']])
display(papers_slow_fastmatched2021_journal.describe()[['c_history_sum_2018','c_history_sum','jif']])
display(papers_fast_slowmatched2021_journal.describe()[['c_history_sum_2018','c_history_sum','jif']])

,c_history_sum_2018,c_history_sum,jif
count,40101.000000,40101.000000,40101.000000
mean,9.854168,40.474402,3.720505
std,4.492502,15.773760,2.623310
min,0.000000,5.000000,0.000000
25%,7.000000,30.000000,2.059603
50%,9.000000,38.000000,3.331395
75%,12.000000,49.000000,4.803213
max,23.000000,296.000000,37.781431


,c_history_sum_2018,c_history_sum,jif
count,40101.000000,40101.000000,40101.000000
mean,9.854168,22.041071,3.720505
std,4.492502,10.239807,2.623310
min,0.000000,0.000000,0.000000
25%,7.000000,15.000000,2.059603
50%,9.000000,21.000000,3.331395
75%,12.000000,29.000000,4.803213
max,23.000000,56.000000,37.781431


,c_history_sum_2018,c_history_sum,jif
count,20561.000000,20561.000000,20561.000000
mean,10.822334,41.449443,4.536851
std,4.438545,14.194099,2.768293
min,0.000000,5.000000,0.000000
25%,8.000000,32.000000,2.907937
50%,10.000000,40.000000,3.993483
75%,13.000000,49.000000,5.714462
max,23.000000,158.000000,37.781431


,c_history_sum_2018,c_history_sum,jif
count,20561.000000,20561.000000,20561.000000
mean,18.182773,41.449443,4.536851
std,6.763447,14.194099,2.768293
min,3.000000,5.000000,0.000000
25%,13.000000,32.000000,2.907937
50%,17.000000,40.000000,3.993483
75%,22.000000,49.000000,5.714462
max,71.000000,158.000000,37.781431


In [10]:
papers_slow_fastmatched2021_journal['jif'].mean()

4.536850615766816

In [11]:
papers_fast_slowmatched2021_journal['jif'].mean()

4.536850615766816